TASK 1 - Managing Conversation History with Summarization


In [1]:
!pip install --quiet openai

In [9]:
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "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"   # Groq API key
os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"

client = OpenAI()

In [10]:
# 2. Conversation Storage

conversation_history = []
run_counter = 0

def add_message(role, content):
    """Add user/assistant/system message to history"""
    conversation_history.append({"role": role, "content": content})

def print_history():
    """Print conversation history"""
    print("\n--- Conversation History ---")
    for m in conversation_history:
        print(f"{m['role']}: {m['content']}")

In [11]:

# 3. Summarization Function

def summarize_history(history):
    """Summarize entire conversation so far using Groq API"""
    messages = [{"role": m["role"], "content": m["content"]} for m in history]
    completion = client.chat.completions.create(
        model="moonshotai/kimi-k2-instruct-0905",   # Used Moonshotai model (Groq-supported model)
        messages=[
            {"role": "system", "content": "Summarize the following conversation concisely."},
            *messages
        ],
        max_tokens=100
    )
    return completion.choices[0].message.content.strip()

In [12]:
# 4. Truncation Options

def truncate_by_turns(n):
    """Keep only the last n conversation turns"""
    global conversation_history
    conversation_history = conversation_history[-n:]

def truncate_by_length(max_chars):
    """Keep only messages within max character length"""
    global conversation_history
    result, total_len = [], 0
    for m in reversed(conversation_history):
        if total_len + len(m["content"]) <= max_chars:
            result.insert(0, m)
            total_len += len(m["content"])
        else:
            break
    conversation_history = result

In [13]:
# 5. Periodic Summarization

def on_new_run(role, content, k=3):
    """
    Add message, increment run counter,
    summarize every k-th run, replace history with summary + last message
    """
    global run_counter, conversation_history

    add_message(role, content)
    run_counter += 1

    if run_counter % k == 0:
        print("\n[Summarizing conversation...]")
        summary = summarize_history(conversation_history)
        conversation_history = [{"role": "system", "content": f"SUMMARY -> {summary}"}]
        conversation_history.append({"role": role, "content": content})

    print_history()

In [14]:
# 6. Demonstration

print("=== Demo: Conversation Flow ===")


on_new_run("user", "Hi, I'm Rahul. Can you help me with my project?")
on_new_run("assistant", "Of course, Rahul. What is the project about?")
on_new_run("user", "It's about conversation summarization using Groq API.")  # triggers summarization at 3rd run
on_new_run("assistant", "Got it. We need to store history and make summaries.")
on_new_run("user", "Yes, also truncate messages when it gets too long.")

print("\n=== Demo: Truncation by last 2 turns ===")
truncate_by_turns(2)
print_history()

print("\n=== Demo: Truncation by 50 characters ===")
truncate_by_length(50)
print_history()


=== Demo: Conversation Flow ===

--- Conversation History ---
user: Hi, I'm Rahul. Can you help me with my project?

--- Conversation History ---
user: Hi, I'm Rahul. Can you help me with my project?
assistant: Of course, Rahul. What is the project about?

[Summarizing conversation...]

--- Conversation History ---
system: SUMMARY -> Got it—you're building a conversation-summarizer that calls Groq's API. How far along are you, and where are you stuck?
user: It's about conversation summarization using Groq API.

--- Conversation History ---
system: SUMMARY -> Got it—you're building a conversation-summarizer that calls Groq's API. How far along are you, and where are you stuck?
user: It's about conversation summarization using Groq API.
assistant: Got it. We need to store history and make summaries.

--- Conversation History ---
system: SUMMARY -> Got it—you're building a conversation-summarizer that calls Groq's API. How far along are you, and where are you stuck?
user: It's about conve

TASK 2 : JSON Schema Classification & Information Extraction

In [16]:
from openai import OpenAI
import os, json
from jsonschema import validate, ValidationError

os.environ["OPENAI_API_KEY"] = "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"   # Groq API key
os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"

client = OpenAI()


In [17]:
# JSON Schema for extracting details

user_info_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}


In [18]:
def extract_user_info(conversation_text):
    response = client.chat.completions.create(
        model="moonshotai/kimi-k2-instruct-0905",   # Groq supported model
        messages=[
            {"role": "system", "content": "Extract user details into structured JSON."},
            {"role": "user", "content": conversation_text}
        ],
        functions=[
            {
                "name": "extract_user_info",
                "description": "Extracts personal details from conversation",
                "parameters": user_info_schema
            }
        ],
        function_call={"name": "extract_user_info"}
    )

    args = response.choices[0].message.function_call.arguments
    return json.loads(args)


In [19]:
def validate_extracted(data):
    try:
        validate(instance=data, schema=user_info_schema)
        return True, None
    except ValidationError as e:
        return False, str(e)


In [20]:
sample_chats = [
    "Hi, I’m Rahul Sharma. My email is rahul@example.com, phone number is 9876543210, I live in Bangalore, and I’m 25 years old.",
    "Hello, this is Ananya Singh, I’m 30. You can reach me at ananya@gmail.com or call me on 9123456789. I stay in Mumbai.",
    "Hey! My name is Arjun Mehta, I’m 22 years old. Email: arjunm22@yahoo.com, phone: 9988776655, and I’m from Delhi."
]

for i, chat in enumerate(sample_chats, 1):
    print(f"\n=== Sample Chat {i} ===")
    extracted = extract_user_info(chat)
    is_valid, error = validate_extracted(extracted)

    print("Extracted JSON:", extracted)
    print("Validation:", "PASS ✅" if is_valid else f"FAIL ❌ - {error}")



=== Sample Chat 1 ===
Extracted JSON: {'age': 25, 'email': 'rahul@example.com', 'location': 'Bangalore', 'name': 'Rahul Sharma', 'phone': '9876543210'}
Validation: PASS ✅

=== Sample Chat 2 ===
Extracted JSON: {'age': 30, 'email': 'ananya@gmail.com', 'location': 'Mumbai', 'name': 'Ananya Singh', 'phone': '9123456789'}
Validation: PASS ✅

=== Sample Chat 3 ===
Extracted JSON: {'age': 22, 'email': 'arjunm22@yahoo.com', 'location': 'Delhi', 'name': 'Arjun Mehta', 'phone': '9988776655'}
Validation: PASS ✅
